In [2]:
import os
import json
import csv
import re
import ftfy
import time
import pandas as pd
import pubchempy as pcp

from openai import OpenAI
from anthropic import Anthropic

from fuzzywuzzy import process
from tqdm import tqdm
from dotenv import load_dotenv
tqdm.pandas()
load_dotenv() 



True

Shared substances of interest. Flattened, removed duplicates, etc.

In [2]:
substances_of_interest_file = r"C:\Users\James\OneDrive\Kansas State University\CIS 830\Project_SparKG\data\drugs_of_interest_list.csv"
substances_of_interest = pd.read_csv(substances_of_interest_file, encoding="utf-8-sig")
substances_of_interest

,drug
0,"2,6-Xylidine"
1,2-amino-5-chloropyridine
2,2-fluoro-2-oxo PCE
3,2-Oxo-3-hydroxy-LSD
4,3-hydroxy flubromazepam
...,...
72,Speciociliatine
73,Temazepam
74,Xylazine
75,Zolpidem


PubChem Synonym set

In [3]:
synonyms = pcp.get_synonyms("2,6-Xylidine", "name")
print(synonyms[0]["CID"])
print(synonyms[0]["Synonym"])

6896
['2,6-Dimethylaniline', '87-62-7', '2,6-XYLIDINE', 'o-Xylidine', '2-Amino-m-xylene', '2,6-Dimethylbenzenamine', 'Benzenamine, 2,6-dimethyl-', '2-Amino-1,3-dimethylbenzene', '2-Amino-1,3-xylene', '2,6-Dimethylphenylamine', '2,6-Xylylamine', '1-Amino-2,6-dimethylbenzene', 'Aniline, 2,6-dimethyl-', 'NCI-C56188', 'vic-m-xylidine', 'NSC 7098', 'DTXSID8026307', '2,6-Dimethyl-aniline', 'MFCD00007747', '4FT62OX08D', 'CHEBI:28738', '2,6-DMA', 'NSC-7098', 'DTXCID006307', 'CAS-87-62-7', 'CCRIS 2373', 'HSDB 2094', 'Benzene, 2-amino-1,3-dimethyl-', 'EINECS 201-758-7', 'BRN 0636332', 'UNII-4FT62OX08D', 'AI3-52358', '2,6 dimethylaniline', '2.6-dimethylaniline', 'Aniline,6-dimethyl-', '2,6 dimethyl aniline', '2,6-dimethyl aniline', 'Benzenamine,6-dimethyl-', '2,6-dimethylbenzeneamine', '2,6-dimethyl-phenylamine', '2,6-dimethyl benzeneamine', '(2,6-dimethylphenyl)amine', 'EC 201-758-7', 'XYLIDENE, 2,6-', '2,6-Dimethylaniline, 99%', '4-12-00-02521 (Beilstein Handbook Reference)', 'SCHEMBL106244', '

In [4]:
substances_of_interest["CID"] = None
substances_of_interest["pubchem_synonyms"] = None

def fetch_pubchem_info(drug_name):
    try:
        results = pcp.get_synonyms(drug_name, "name")
        if results:
            cid = results[0].get("CID", None)
            synonyms = results[0].get("Synonym", None)
            return cid, synonyms
    except Exception as e:
        print(f"Error retrieving data for '{drug_name}': {e}")
    return None, None

# Iterate through each row of the DataFrame and add PubChem data
for index, row in substances_of_interest.iterrows():
    drug_name = row["drug"].strip()  
    cid, synonyms = fetch_pubchem_info(drug_name)
    substances_of_interest.at[index, "CID"] = cid
    substances_of_interest.at[index, "pubchem_synonyms"] = synonyms

substances_of_interest

,drug,CID,pubchem_synonyms
0,"2,6-Xylidine",6896,"[2,6-Dimethylaniline, 87-62-7, 2,6-XYLIDINE, o..."
1,2-amino-5-chloropyridine,66174,"[2-Amino-5-chloropyridine, 1072-98-6, 5-chloro..."
2,2-fluoro-2-oxo PCE,None,None
3,2-Oxo-3-hydroxy-LSD,10155149,"[2-Oxo-3-hydroxy-lysergide, 2-Oxo-3-hydroxy-LS..."
4,3-hydroxy flubromazepam,13126012,"[3-hydroxy Flubromazepam, 3-Hydroxyflubromazep..."
...,...,...,...
72,Speciociliatine,15560576,"[Speciociliatine, 14382-79-7, Speciociliatin, ..."
73,Temazepam,5391,"[temazepam, Restoril, Hydroxydiazepam, Methylo..."
74,Xylazine,5707,"[xylazine, 7361-61-7, N-(2,6-Dimethylphenyl)-5..."
75,Zolpidem,5732,"[zolpidem, 82626-48-0, Zolpidemum, Zolpidemum ..."


In [5]:
none_results = substances_of_interest[
    substances_of_interest["CID"].isnull() | substances_of_interest["pubchem_synonyms"].isnull()]

print(none_results)

                                   drug   CID pubchem_synonyms
2                    2-fluoro-2-oxo PCE  None             None
5   3-hydroxy flubromazepam glucuronide  None             None
7                               4-HIAA   None             None
11                 7-OH-CBD glucuronide  None             None
13                          8R-OH-R-HHC  None             None
14                          8S-OH-R-HHC  None             None
16              alpha-hydroxybromazolam  None             None
25                     delta-8-THC-COOH  None             None
26                     delta-9-THC-COOH  None             None
37                                  MDA  None             None
49               N-desethylmetonitazene  None             None
57             N-Pyrrolidinoetonitazene  None             None
64               para-Fluoronorfentanyl  None             None
70                           R-HHC-COOH  None             None
71                           S-HHC-COOH  None          

In [ ]:
hand_picked_IDs_dictionary = {
  "2-fluoro-2-oxo PCE": 168323041,
  "3-hydroxy flubromazepam glucuronide": 13126012,
  "4-HIAA ": 7061393,
  "7-OH-CBD glucuronide": 11301963,
  "8R-OH-R-HHC": 168323034,
#   "8S-OH-R-HHC": None,
#   "alpha-hydroxybromazolam": None,
  "delta-8-THC-COOH": 162389,
  "delta-9-THC-COOH": 107885,
  "MDA": 1614,
#   "N-desethylmetonitazene": "",
  "N-Pyrrolidinoetonitazene": 155804760,
  "para-Fluoronorfentanyl": 10610682,
#   "R-HHC-COOH": "R-hexahydrocannabinol carboxylic acid",
#   "S-HHC-COOH": "S-hexahydrocannabinol carboxylic acid"
}

for drug, hand_picked_ID in hand_picked_IDs_dictionary.items():
    hand_picked_ID = hand_picked_IDs_dictionary[drug]
    compound = pcp.Compound.from_cid(hand_picked_ID)
    synonyms = compound.synonyms
    matching_rows = substances_of_interest[substances_of_interest["drug"] == drug]
    if not matching_rows.empty:
        idx = matching_rows.index[0]
        
        substances_of_interest.at[idx, "CID"] = hand_picked_ID
        substances_of_interest.at[idx, "pubchem_synonyms"] = synonyms

In [7]:
none_results = substances_of_interest[
    substances_of_interest["CID"].isnull() | substances_of_interest["pubchem_synonyms"].isnull()]

print(none_results)

                       drug   CID pubchem_synonyms
14              8S-OH-R-HHC  None             None
16  alpha-hydroxybromazolam  None             None
49   N-desethylmetonitazene  None             None
70               R-HHC-COOH  None             None
71               S-HHC-COOH  None             None


In [ ]:


client = Anthropic(api_key=os.getenv('anthropic-first-key'))

for drug in none_results["drug"]:

    prompt = f"""As a chemistry database expert, I need identification information for {drug}.
                Please provide The official PubChem name used to identify the compound {drug}
                that can be used for the PubChem API such as
                "
                import pubchempy as pcp
                results = pcp.get_synonyms(drug_name, "name")
                "
                return the exact name with no other text. If you cannot find a match, return "None" with no other text."""

    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=1000,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    print(response.content[0].text)




None
8-bromo-1-methyl-6-phenyl-4H-[1,2,4]triazolo[4,3-a][1,4]benzodiazepine-4,5-diol
2-[2-[[1-(3-methoxyphenyl)propan-2-yl]amino]ethyl]-5-nitro-1H-benzimidazole
None
(S)-7-Carboxy-Δ8-tetrahydrocannabinol


Prompt given by Dr. Hsu in 830 class

In [9]:

client = OpenAI(api_key= os.getenv('First_Key') )

response = client.responses.create(
    model="o3-mini-2025-01-31", #"gpt-4o-mini""o3-mini"
    input="For each of the quasi-psychedelic phytocannabinoids listed at " \
    "https://m.psychonautwiki.org/wiki/Psychoactive_substance_index , " \
    "provide the following in Json format: " \
    "Name, Known Synonyms, Pharmacology, Effects,Toxicity and Harm Potential, and Legal Status"
)

print(response.output_text)


Below is one possible JSON‐representation covering the three main quasi‐psychedelic cannabinoids that occur naturally in cannabis. (Note that details on these substances vary between sources; the entries below are summaries based on the information collected on PsychonautWiki and related literature and are meant to serve as an example overview rather than an exhaustive, authoritative review.)

[
  {
    "Name": "Δ9-Tetrahydrocannabinol (Δ9-THC)",
    "Known_Synonyms": [
      "THC",
      "Delta-9 THC",
      "Δ9-THC"
    ],
    "Pharmacology": "Acts as a partial agonist at both CB1 and CB2 cannabinoid receptors. It modulates neurotransmitter release throughout the brain and is thought to interact with various non-cannabinoid receptor systems (e.g., GPR55). This receptor action underlies its psychoactive and analgesic properties.",
    "Effects": "Produces euphoria, altered sensory perception, digital and time distortion, introspection, and in some cases anxiety or paranoia at higher d

In [ ]:

new_columns = [
    "psychonautwiki_Name", 
    "psychonautwiki_Synonyms", 
    "Pharmacology", 
    "Effects", 
    "Toxicity and Harm Potential", 
    "Legal Status"
]

for col in new_columns:
    substances_of_interest[col] = None

for index, substance in substances_of_interest.iterrows():
    drug_name = substance["drug"]
    CID = substance["CID"]
    pubchem_synonyms = substance["pubchem_synonyms"]

    prompt = (
        f"""Determine if {drug_name} is a substance on this website https://m.psychonautwiki.org/wiki/Psychoactive_substance_index, 
        {drug_name} has a PubChem ID {CID} and the following synonyms: {pubchem_synonyms}
        If there is information on psychonautwiki about this substance, provide the following information in Json format: 
        psychonautwiki_Name, psychonautwiki_Synonyms, Pharmacology, Effects, Toxicity and Harm Potential, and Legal Status. 
        DO NOT provide duplicate synonyms! 
        If psychonautwiki does not have information about {drug_name}, return just the word None"""
    )

    response = client.responses.create(
        model="o3-mini-2025-01-31",  
        input=prompt
    )
    output_text = response.output_text.strip()
    
    try:
        # if API returned "None". all new columns None.
        if output_text.lower() == "none":
            for col in new_columns:
                substances_of_interest.at[index, col] = None
        else:
            result = json.loads(output_text)
            if isinstance(result, dict):
                for col in new_columns:
                    substances_of_interest.at[index, col] = result.get(col, None)
            else:
                for col in new_columns:
                    substances_of_interest.at[index, col] = None

    except Exception as e:
        for col in new_columns:
            substances_of_interest.at[index, col] = None
        print(f"Error processing substance {drug_name}: {e}")


In [11]:
substances_of_interest.info(verbose=True)
substances_of_interest

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   drug                         77 non-null     object
 1   CID                          72 non-null     object
 2   pubchem_synonyms             72 non-null     object
 3   psychonautwiki_Name          38 non-null     object
 4   psychonautwiki_Synonyms      38 non-null     object
 5   Pharmacology                 38 non-null     object
 6   Effects                      38 non-null     object
 7   Toxicity and Harm Potential  38 non-null     object
 8   Legal Status                 38 non-null     object
dtypes: object(9)
memory usage: 5.5+ KB


,drug,CID,pubchem_synonyms,psychonautwiki_Name,psychonautwiki_Synonyms,Pharmacology,Effects,Toxicity and Harm Potential,Legal Status
0,"2,6-Xylidine",6896,"[2,6-Dimethylaniline, 87-62-7, 2,6-XYLIDINE, o...",None,None,None,None,None,None
1,2-amino-5-chloropyridine,66174,"[2-Amino-5-chloropyridine, 1072-98-6, 5-chloro...",None,None,None,None,None,None
2,2-fluoro-2-oxo PCE,168323041,"[2-fluoro-2-oxo PCE (hydrochloride), 2850352-6...",None,None,None,None,None,None
3,2-Oxo-3-hydroxy-LSD,10155149,"[2-Oxo-3-hydroxy-lysergide, 2-Oxo-3-hydroxy-LS...",None,None,None,None,None,None
4,3-hydroxy flubromazepam,13126012,"[3-hydroxy Flubromazepam, 3-Hydroxyflubromazep...",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...
72,Speciociliatine,15560576,"[Speciociliatine, 14382-79-7, Speciociliatin, ...",None,None,None,None,None,None
73,Temazepam,5391,"[temazepam, Restoril, Hydroxydiazepam, Methylo...",Temazepam,"[Restoril, Temtabs, Normison, Temaz]",Temazepam is a benzodiazepine that acts as a p...,"Common effects include pronounced sedation, re...","Temazepam, like other benzodiazepines, carries...",Temazepam is a prescription medication in most...
74,Xylazine,5707,"[xylazine, 7361-61-7, N-(2,6-Dimethylphenyl)-5...",Xylazine,"[Rompun, Xilazina, Xylazin, Chanazine, Xylazinum]",Xylazine acts primarily as an α2-adrenergic re...,"Users report sedation, marked drowsiness, anal...",Xylazine has a high toxicity risk when used ou...,Xylazine is not approved for human use and is ...
75,Zolpidem,5732,"[zolpidem, 82626-48-0, Zolpidemum, Zolpidemum ...",Zolpidem,"[Ambien, Stilnox]",Zolpidem is a nonbenzodiazepine hypnotic that ...,"At therapeutic doses, zolpidem induces rapid s...",While effective for short-term treatment of in...,Zolpidem is a prescription drug and is classif...


In [12]:

DEA_drug_slang = pd.read_csv('drug_slang.csv', encoding='utf-8-sig')
DEA_drug_slang


,index_term,alt_term
0,Acetaminophen and Oxycodone Combination (Perco...,512s; Bananas; Blue; Blue Dynamite; Blueberrie...
1,Alprazolam (Xanax®),Bars; Benzos; Bicycle Handle Bars; Bicycle Par...
2,Amphetamine,Acelerador; Amy; Amps; Bam; B-Bombs; Beans; Be...
3,Amphetamine and Dextroamphetamine Combination ...,A-Train; Abby; Addy; Amps; Christmas Trees; Co...
4,Buprenorphine and Naloxone Combination (Suboxo...,"Boxes, Bupes; Oranges; Sobos; Stop Signs; Stop..."
5,Clonazepam (Klonopin®),Benzos; K; K-Pin; Pin; Super Valium; Tranks
6,Cocaine,7; 62; 77; 777; 921; A-1; Adidas; All-American...
7,Crack Cocaine,51s; 151s; 501s; Apple Jack; Baby T; Base; Bas...
8,Ecstasy/MDMA/Molly,Adam; Baby Slits; Beans; Blue Kisses; Blue Sup...
9,Fentanyl and Fentanyl Derivatives,Apache; Birria (fentanyl mixed with heroin); B...


In [ ]:
for index, row in DEA_drug_slang.iterrows():
    index_term = row["index_term"]  
    # Find the top three closest matches 
    top_matches = process.extract(index_term, substances_of_interest["drug"].tolist(), limit=3)
    print("------------------------------------------------------------")
    for match, score in top_matches:
        if score > 50:  ################################## 
            print(f"{index_term} | {match} | {score}")

#I skimmed through these 2 find a threshold make sure the matches are good.

------------------------------------------------------------
Acetaminophen and Oxycodone Combination (Percocet®) | Oxycodone | 90
Acetaminophen and Oxycodone Combination (Percocet®) | Noroxycodone | 75
Acetaminophen and Oxycodone Combination (Percocet®) | Ketamine | 68
------------------------------------------------------------
Alprazolam (Xanax®) | Bromazolam | 63
Alprazolam (Xanax®) | Lorazepam | 60
------------------------------------------------------------
Amphetamine | Amphetamine | 100
Amphetamine | Methamphetamine | 85
Amphetamine | Ketamine | 74
------------------------------------------------------------
Amphetamine and Dextroamphetamine Combination (Adderall®) | Amphetamine | 90
Amphetamine and Dextroamphetamine Combination (Adderall®) | Ketamine | 79
Amphetamine and Dextroamphetamine Combination (Adderall®) | Methamphetamine | 72
------------------------------------------------------------
Buprenorphine and Naloxone Combination (Suboxone®) | Buprenorphine | 90
Buprenorphin

In [ ]:

substances_of_interest["DEA_name"] = None
substances_of_interest["DEA_synonyms"] = None
for index, row in DEA_drug_slang.iterrows():
    index_term = row["index_term"]  
    alt_term = row["alt_term"]
    
    # Find the closest match in the substances_of_interest DataFrame
    match, score = process.extractOne(index_term, substances_of_interest["drug"].tolist())
    
    if score > 88:  ############################### 
        matched_index = substances_of_interest[substances_of_interest["drug"] == match].index[0]
        substances_of_interest.at[matched_index, "DEA_name"] = index_term
        substances_of_interest.at[matched_index, "DEA_synonyms"] = alt_term
    else:
        print(index_term)


# DEA names without match to our data set

Alprazolam (Xanax®)
Clonazepam (Klonopin®)
Cocaine
Crack Cocaine
Flunitrazepam (Rohypnol®)
GHB (Gamma-Hydroxybutyric Acid)
Heroin
Khat
LSD (Lysergic Acid Diethylamide)
Marijuana
Marijuana Concentrates/Hash Oil
Mescaline/Peyote
Methylphenidate (Ritalin®, Concerta®, Daytrana®)
Opium
Promethazine
Synthetic Cannabinoids
Synthetic Cathinones
Steroids
U-47700


In [15]:
# fix 3

# Amphetamine and Dextroamphetamine Combination (Adderall®) | Amphetamine | 90
substances_of_interest.loc[substances_of_interest['drug']=='Amphetamine', "DEA_name"] = None
substances_of_interest.loc[substances_of_interest['drug']=='Amphetamine', "DEA_synonyms"] = None

# Buprenorphine and Naloxone Combination (Suboxone®) | Buprenorphine | 90
substances_of_interest.loc[substances_of_interest['drug']=='Buprenorphine', "DEA_name"] = None
substances_of_interest.loc[substances_of_interest['drug']=='Buprenorphine', "DEA_synonyms"] = None

# LSD (Lysergic Acid Diethylamide) | LSD | 60
substances_of_interest.loc[substances_of_interest['drug']=='LSD', "DEA_name"] = "LSD (Lysergic Acid Diethylamide)"
substances_of_interest.loc[substances_of_interest['drug']=='LSD', "DEA_synonyms"] = DEA_drug_slang.loc[DEA_drug_slang['index_term'] == 'LSD (Lysergic Acid Diethylamide)', 'alt_term'].iloc[0]

substances_of_interest.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   drug                         77 non-null     object
 1   CID                          72 non-null     object
 2   pubchem_synonyms             72 non-null     object
 3   psychonautwiki_Name          38 non-null     object
 4   psychonautwiki_Synonyms      38 non-null     object
 5   Pharmacology                 38 non-null     object
 6   Effects                      38 non-null     object
 7   Toxicity and Harm Potential  38 non-null     object
 8   Legal Status                 38 non-null     object
 9   DEA_name                     11 non-null     object
 10  DEA_synonyms                 11 non-null     object
dtypes: object(11)
memory usage: 6.7+ KB


In [16]:
DEA_drug_slang[DEA_drug_slang['index_term'] == 'LSD (Lysergic Acid Diethylamide)']['alt_term']

17    Aceite; Acelide; Acid; Acido; Alice; Angels in...
Name: alt_term, dtype: object

In [17]:
substances_of_interest[substances_of_interest['drug']=='LSD']["DEA_synonyms"]

36    Aceite; Acelide; Acid; Acido; Alice; Angels in...
Name: DEA_synonyms, dtype: object

In [18]:
lexicon_csv = r"C:\Users\James\OneDrive\Kansas State University\CIS 830\Project_SparKG\code\data_processing\drugs_of_abuse_lexicon.csv"
lexicon = pd.read_csv(lexicon_csv, encoding="utf-8-sig")
lexicon 

,index term,DrugBank ID,widely discussed,GPT-3 synonyms
0,bromazepam,DB01558,True,"'bromaze','lexotan','bromazolam','bromaz','zep..."
1,dexfenfluramine,DB01191,False,"'appetite-decreasing_medication','fenfen','fen..."
2,oxymorphone,DB01192,True,"'endocet','oxymorphone_hydrochloride','numorph..."
3,methylphenidate,DB00422,True,"'ritalin','concerta','metadate','attention_def..."
4,methamphetamine,DB01577,True,"'methamphet','methamphetamine_hydrochloride','..."
...,...,...,...,...
93,alprazolam,DB00404,True,"'xanax','a-prazolam','a-pra-zolam','alprazoram..."
94,fenfluramine,DB00574,False,"'plegine','levofenfluramine','ionamin','fen-ph..."
95,zolpidem,DB00425,False,"'zolpidem_tartrate','zolpidem_10_mg','zolpidem..."
96,modafinil,DB00745,True,"'moadfinil','modifanil','moddafinil','modafini..."


In [19]:
for index, row in lexicon.iterrows():
    index_term = row["index term"]  
    DrugBank_ID = row["DrugBank ID"]
    widely_discussed = row["widely discussed"]
    GPT_synonyms = row["GPT-3 synonyms"]
    # Find the top three closest matches 
    top_matches = process.extract(index_term, substances_of_interest["drug"].tolist(), limit=3)
    print("------------------------------------------------------------")
    for match, score in top_matches:
        if score > 88:  ################################## 
            print(f"{index_term} | {match} | {score}")

------------------------------------------------------------
bromazepam | 3-hydroxy flubromazepam | 90
bromazepam | 3-hydroxy flubromazepam glucuronide | 90
------------------------------------------------------------
------------------------------------------------------------
oxymorphone | Oxymorphone | 100
------------------------------------------------------------
------------------------------------------------------------
methamphetamine | Methamphetamine | 100
------------------------------------------------------------
------------------------------------------------------------
morphine | Morphine | 100
morphine | 6-acetylmorphine | 90
------------------------------------------------------------
phencyclidine | Phencyclidine | 100
------------------------------------------------------------
dihydrocodeine | Codeine | 90
------------------------------------------------------------
------------------------------------------------------------
------------------------------------

In [20]:
substances_of_interest["DrugBank_ID"] = None
substances_of_interest["widely_discussed"] = None
substances_of_interest["GPT_synonyms"] = None
for index, row in lexicon.iterrows():
    index_term = row["index term"]  
    DrugBank_ID = row["DrugBank ID"]
    widely_discussed = row["widely discussed"]
    GPT_synonyms = row["GPT-3 synonyms"]
    # Find the top 
    match, score = process.extractOne(index_term, substances_of_interest["drug"].tolist())
    
    if score > 92:  ############################### 
        matched_index = substances_of_interest[substances_of_interest["drug"] == match].index[0]
        substances_of_interest.at[matched_index, "DrugBank_ID"] = DrugBank_ID
        substances_of_interest.at[matched_index, "widely_discussed"] = widely_discussed
        substances_of_interest.at[matched_index, "GPT_synonyms"] = GPT_synonyms

substances_of_interest.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   drug                         77 non-null     object
 1   CID                          72 non-null     object
 2   pubchem_synonyms             72 non-null     object
 3   psychonautwiki_Name          38 non-null     object
 4   psychonautwiki_Synonyms      38 non-null     object
 5   Pharmacology                 38 non-null     object
 6   Effects                      38 non-null     object
 7   Toxicity and Harm Potential  38 non-null     object
 8   Legal Status                 38 non-null     object
 9   DEA_name                     11 non-null     object
 10  DEA_synonyms                 11 non-null     object
 11  DrugBank_ID                  22 non-null     object
 12  widely_discussed             22 non-null     object
 13  GPT_synonyms                 22 non-n

In [21]:
redmed_lexicon_csv = r"C:\Users\James\OneDrive\Kansas State University\CIS 830\Project_SparKG\data\redmed_lexicon.csv"
redmed_lexicon = pd.read_csv(redmed_lexicon_csv, encoding="utf-8-sig")
redmed_lexicon.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997 entries, 0 to 2996
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   dbid             2997 non-null   object
 1   drug             2997 non-null   object
 2   known            2975 non-null   object
 3   misspellingPhon  484 non-null    object
 4   edOne            1317 non-null   object
 5   edTwo            1519 non-null   object
 6   pillMark         19 non-null     object
 7   google_ms        28 non-null     object
 8   google_title     702 non-null    object
 9   google_snippet   727 non-null    object
 10  ud_slang         40 non-null     object
 11  missed           2856 non-null   object
dtypes: object(12)
memory usage: 281.1+ KB


In [22]:

substances_of_interest["dbid"] = None
substances_of_interest["known"] = None
substances_of_interest["misspellingPhon"] = None
substances_of_interest["edOne"] = None
substances_of_interest["edTwo"] = None
substances_of_interest["pillMark"] = None
substances_of_interest["google_ms"] = None
substances_of_interest["google_title"] = None
substances_of_interest["google_snippet"] = None
substances_of_interest["ud_slang"] = None
substances_of_interest["missed"] = None
for index, row in redmed_lexicon.iterrows():
    drug_redmed_lexicon = row["drug"]  
    if drug_redmed_lexicon == "etonitazene":
        continue
    
    dbid = row["dbid"]
    known = row["known"]
    misspellingPhon = row["misspellingPhon"]
    edOne = row["edOne"]
    edTwo = row["edTwo"]
    pillMark = row["pillMark"]
    google_ms = row["google_ms"]
    google_title = row["google_title"]
    google_snippet = row["google_snippet"]
    ud_slang = row["ud_slang"]
    missed = row["missed"]
    
    # Find the top 
    match, score = process.extractOne(drug_redmed_lexicon, substances_of_interest["drug"].tolist())
    
    if score > 92:  ############################### 
        matched_index = substances_of_interest[substances_of_interest["drug"] == match].index[0]
        substances_of_interest.at[matched_index, "dbid"] = dbid
        substances_of_interest.at[matched_index, "known"] = known
        substances_of_interest.at[matched_index, "misspellingPhon"] = misspellingPhon
        substances_of_interest.at[matched_index, "edOne"] = edOne
        substances_of_interest.at[matched_index, "edTwo"] = edTwo
        substances_of_interest.at[matched_index, "pillMark"] = pillMark
        substances_of_interest.at[matched_index, "google_ms"] = google_ms
        substances_of_interest.at[matched_index, "google_title"] = google_title
        substances_of_interest.at[matched_index, "google_snippet"] = google_snippet
        substances_of_interest.at[matched_index, "ud_slang"] = ud_slang
        substances_of_interest.at[matched_index, "missed"] = missed
        if score < 100:
            print("------------------------------------------------------------")
            print(match)
            print(drug_redmed_lexicon)
            print(score)
    
substances_of_interest.info(verbose=True)

------------------------------------------------------------
Psilocybin
psilocybine
95
------------------------------------------------------------
Nordiazepam
nordazepam
95
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   drug                         77 non-null     object
 1   CID                          72 non-null     object
 2   pubchem_synonyms             72 non-null     object
 3   psychonautwiki_Name          38 non-null     object
 4   psychonautwiki_Synonyms      38 non-null     object
 5   Pharmacology                 38 non-null     object
 6   Effects                      38 non-null     object
 7   Toxicity and Harm Potential  38 non-null     object
 8   Legal Status                 38 non-null     object
 9   DEA_name                     11 non-null     object
 10  DEA_synonyms                 11 no

In [23]:
substances_of_interest.to_csv(r"C:\Users\James\OneDrive\Kansas State University\CIS 830\Project_SparKG\data\drugs_with_data2.csv", index=False, encoding="utf-8-sig")

In [29]:
# Initialize the column to store the synonyms
substances_of_interest["most_likely_synonyms"] = None

# Instantiate the client (ensure your API key is correctly loaded)
client = OpenAI(api_key=os.getenv('First_Key'))

# Iterate over each row in the DataFrame
for index, row in substances_of_interest.iterrows():
    # Extract the drug value explicitly
    drug = row["drug"]
    
    prompt = f"""Based on the following information, provide up to 20 most probable synonyms for the drug {drug}:
    pubchem_synonyms: {row["pubchem_synonyms"]}
    psychonautwiki_Name: {row["psychonautwiki_Name"]}
    psychonautwiki_Synonyms: {row["psychonautwiki_Synonyms"]}
    Pharmacology: {row["Pharmacology"]}
    Effects: {row["Effects"]}
    Toxicity and Harm Potential: {row["Toxicity and Harm Potential"]}
    Legal Status: {row["Legal Status"]}
    DAE_name: {row["DEA_name"]}
    DEA_synonyms: {row["DEA_synonyms"]}
    widely_discussed: {row["widely_discussed"]}
    GPT_synonyms: {row["GPT_synonyms"]}
    known: {row["known"]}
    misspellingPhon: {row["misspellingPhon"]}
    edOne: {row["edOne"]}
    edTwo: {row["edTwo"]}
    pillMark: {row["pillMark"]}
    google_ms: {row["google_ms"]}
    google_title: {row["google_title"]}
    google_snippet: {row["google_snippet"]}
    ud_slang: {row["ud_slang"]}
    missed: {row["missed"]}

    Do not provide duplicates or any other text. Just return the synonyms in a list format, separated by commas.
    If you cannot find any synonyms, return "None" with no other text.
    """
    
    # Get response from the language model
    response = client.responses.create(
        model="o3-mini-2025-01-31",  # Make sure this model name is valid for your setup
        input=prompt
    )

    output_text = response.output_text.strip()
    
    if output_text.lower() == "none":
        output_text = None
        print(f"No info for '{drug}'")
    else:
        output_text = ftfy.fix_text(output_text)
    
    # Update the DataFrame entry; using index here (or 'drug' if the DataFrame is indexed by drug)
    substances_of_interest.at[index, "most_likely_synonyms"] = output_text


No info for '8S-OH-R-HHC'
No info for 'alpha-hydroxybromazolam'
No info for 'N-desethylmetonitazene'
No info for 'R-HHC-COOH'
No info for 'S-HHC-COOH'


In [30]:
substances_of_interest.to_csv(r"C:\Users\James\OneDrive\Kansas State University\CIS 830\Project_SparKG\data\drugs_with_data3.csv", index=False, encoding="utf-8-sig")

In [7]:
import os
import json
import csv
import re
import ftfy
import time
import pandas as pd
import pubchempy as pcp

from openai import OpenAI
from anthropic import Anthropic

from fuzzywuzzy import process
from tqdm import tqdm
from dotenv import load_dotenv
tqdm.pandas()
load_dotenv() 

websites = [
    "https://www.drugbank.ca/drugs/",
    "https://pubchem.ncbi.nlm.nih.gov/",
    "https://www.bluelight.org",
    "https://psychonautwiki.org/wiki/List/substances-new",
    "https://psychonautwiki.org/wiki/List_of_pharmaceuticals",
    "https://psychonautwiki.org/wiki/Psychoactive_substance_index",
    "https://erowid.org/experiences/exp_front.php",
    "https://www.drugabuse.gov/publications/drugfacts",
    "https://www.erowid.org/psychoactives/",
    "https://www.Wikipedia.org",
]

substances_of_interest  = pd.read_csv(r"C:\Users\James\OneDrive\Kansas State University\CIS 830\Project_SparKG\data\drugs_with_data3.csv", encoding="utf-8-sig")

In [8]:
websites = [
    "https://www.drugbank.ca/drugs/",
    "https://pubchem.ncbi.nlm.nih.gov/",
    "https://www.bluelight.org",
    "https://psychonautwiki.org/wiki/List/substances-new",
    "https://psychonautwiki.org/wiki/List_of_pharmaceuticals",
    "https://psychonautwiki.org/wiki/Psychoactive_substance_index",
    "https://erowid.org/experiences/exp_front.php",
    "https://www.drugabuse.gov/publications/drugfacts",
    "https://www.erowid.org/psychoactives/",
    "https://www.Wikipedia.org",
]

for website in websites:
    substances_of_interest[website] = None

# Initialize columns to store website and response results.
substances_of_interest["website"] = None
substances_of_interest["response"] = None

client = Anthropic(api_key=os.getenv('anthropic-first-key'))

# Iterate over each row so we can update by index.
for idx, row in tqdm(substances_of_interest.iterrows(), total=len(substances_of_interest)):
    drug = row["drug"]
    for website in websites:
        prompt = f"""You are tasked with exploring the website "{website}" to extract comprehensive, contextually rich information about the drug "{drug}". Your objectives are to:
                    1. Identify detailed descriptions and contextual indicators regarding the drug, including properties, usage patterns, effects, and associated behaviors.
                    2. Perform term sense disambiguation: if "{drug}" appears in multiple contexts or has several meanings, focus explicitly on those relating to its illicit use as a controlled substance.
                    3. Extract synonyms, slang terms, street names, and related references that could be associated with "{drug}" in social media contexts, linking these to our drug knowledge base.
                    4. Capture any contextual relationships between this drug and other substances, usage patterns, user communities, or distribution methods that appear on the website.
                    5. Present the findings as a clear and simple paragraph without additional commentary or formatting.
                    Possible synonyms for "{drug}" include: {row["most_likely_synonyms"]}.
                    If minimal or no relevant information is found for "{drug}" on this website, do not infer or fabricate details. Instead, return "None" with no other text.
                    This output will serve as contextual training data for our retrieval-augmented generation system focused on detecting illicit drug references in social media text."""
        
        # Use streaming for long operations
        with client.messages.stream(
            model="claude-3-7-sonnet-20250219",
            max_tokens=50000,
            messages=[
                {"role": "user", "content": prompt}
            ]
        ) as stream:
            # Collect the full response from the stream
            output_text = ""
            for text in stream.text_stream:
                output_text += text
        
        # Process the response
        output_text = output_text.strip()
        
        # Access the response content; adjust the following if your API differs.
        # output_text = response.content[0].text.strip()
        if output_text.lower() == "none":
            output_text = None
            print(f"No info for '{drug}' on '{website}'")
        else:
            output_text = ftfy.fix_text(output_text)

        substances_of_interest.at[idx, website] = output_text


  0%|          | 0/77 [00:00<?, ?it/s]

No info for '2,6-Xylidine' on 'https://www.bluelight.org'
No info for '2,6-Xylidine' on 'https://psychonautwiki.org/wiki/List/substances-new'
No info for '2,6-Xylidine' on 'https://psychonautwiki.org/wiki/List_of_pharmaceuticals'
No info for '2,6-Xylidine' on 'https://psychonautwiki.org/wiki/Psychoactive_substance_index'
No info for '2,6-Xylidine' on 'https://erowid.org/experiences/exp_front.php'
No info for '2,6-Xylidine' on 'https://www.drugabuse.gov/publications/drugfacts'
No info for '2,6-Xylidine' on 'https://www.erowid.org/psychoactives/'


  1%|▏         | 1/77 [00:18<23:47, 18.79s/it]

No info for '2,6-Xylidine' on 'https://www.Wikipedia.org'
No info for '2-amino-5-chloropyridine ' on 'https://www.drugbank.ca/drugs/'
No info for '2-amino-5-chloropyridine ' on 'https://www.bluelight.org'
No info for '2-amino-5-chloropyridine ' on 'https://psychonautwiki.org/wiki/List/substances-new'
No info for '2-amino-5-chloropyridine ' on 'https://psychonautwiki.org/wiki/List_of_pharmaceuticals'
No info for '2-amino-5-chloropyridine ' on 'https://psychonautwiki.org/wiki/Psychoactive_substance_index'
No info for '2-amino-5-chloropyridine ' on 'https://erowid.org/experiences/exp_front.php'
No info for '2-amino-5-chloropyridine ' on 'https://www.drugabuse.gov/publications/drugfacts'
No info for '2-amino-5-chloropyridine ' on 'https://www.erowid.org/psychoactives/'


  3%|▎         | 2/77 [00:34<20:56, 16.76s/it]

...

 97%|█████████▋| 75/77 [40:26<00:58, 29.47s/it]

No info for 'Zolpidem ' on 'https://www.drugabuse.gov/publications/drugfacts'


 99%|█████████▊| 76/77 [41:16<00:35, 35.82s/it]

No info for 'Zopiclone' on 'https://www.drugabuse.gov/publications/drugfacts'


100%|██████████| 77/77 [42:10<00:00, 32.87s/it]


In [10]:
substances_of_interest.to_csv(r"C:\Users\James\OneDrive\Kansas State University\CIS 830\Project_SparKG\data\drugs_with_data4.csv", index=False, encoding="utf-8-sig")

In [11]:
substances_of_interest.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 38 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   drug                                                          77 non-null     object 
 1   CID                                                           72 non-null     float64
 2   pubchem_synonyms                                              72 non-null     object 
 3   psychonautwiki_Name                                           38 non-null     object 
 4   psychonautwiki_Synonyms                                       38 non-null     object 
 5   Pharmacology                                                  38 non-null     object 
 6   Effects                                                       38 non-null     object 
 7   Toxicity and Harm Potential                                   38 non-null

In [12]:
substances_of_interest

,drug,CID,pubchem_synonyms,psychonautwiki_Name,psychonautwiki_Synonyms,Pharmacology,Effects,Toxicity and Harm Potential,Legal Status,DEA_name,...,https://www.bluelight.org,https://psychonautwiki.org/wiki/List/substances-new,https://psychonautwiki.org/wiki/List_of_pharmaceuticals,https://psychonautwiki.org/wiki/Psychoactive_substance_index,https://erowid.org/experiences/exp_front.php,https://www.drugabuse.gov/publications/drugfacts,https://www.erowid.org/psychoactives/,https://www.Wikipedia.org,website,response
0,"2,6-Xylidine",6896.0,"['2,6-Dimethylaniline', '87-62-7', '2,6-XYLIDI...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
1,2-amino-5-chloropyridine,66174.0,"['2-Amino-5-chloropyridine', '1072-98-6', '5-c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,2-fluoro-2-oxo PCE,168323041.0,"['2-fluoro-2-oxo PCE (hydrochloride)', '285035...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,I cannot perform real-time web scraping of ext...,None,None,None,None,None,None,None,None,None
3,2-Oxo-3-hydroxy-LSD,10155149.0,"['2-Oxo-3-hydroxy-lysergide', '2-Oxo-3-hydroxy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,I cannot and will not browse or scrape the blu...,Based on my exploration of the provided websit...,"Based on the provided URL and my knowledge, I ...",2-Oxo-3-hydroxy-LSD is not a psychoactive subs...,None,None,2-Oxo-3-hydroxy-LSD is not a primary psychoact...,None,None,None
4,3-hydroxy flubromazepam,13126012.0,"['3-hydroxy Flubromazepam', '3-Hydroxyflubroma...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"I cannot explore or scrape the website ""https:...",None,None,None,None,None,None,I cannot access the Wikipedia website in real-...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Speciociliatine,15560576.0,"['Speciociliatine', '14382-79-7', 'Speciocilia...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Speciociliatine is one of the minor alkaloids ...,None,Speciociliatine is one of the minor alkaloids ...,None,None,Speciociliatine is a minor alkaloid found in t...,Speciociliatine is one of the indole alkaloids...,None,None
73,Temazepam,5391.0,"['temazepam', 'Restoril', 'Hydroxydiazepam', '...",Temazepam,"['Restoril', 'Temtabs', 'Normison', 'Temaz']",Temazepam is a benzodiazepine that acts as a p...,"Common effects include pronounced sedation, re...","Temazepam, like other benzodiazepines, carries...",Temazepam is a prescription medication in most...,NaN,...,I cannot actively browse or search websites in...,Temazepam is a short-to-intermediate-acting be...,Temazepam is a benzodiazepine medication with ...,Temazepam is a benzodiazepine substance belong...,Temazepam is a benzodiazepine medication that ...,None,None,I cannot browse or explore websites in real-ti...,None,None
74,Xylazine,5707.0,"['xylazine', '7361-61-7', 'N-(2,6-Dimethylphen...",Xylazine,"['Rompun', 'Xilazina', 'Xylazin', 'Chanazine',...",Xylazine acts primarily as an α2-adrenergic re...,"Users report sedation, marked drowsiness, anal...",Xylazine has a high toxicity risk when used ou...,Xylazine is not approved for human use and is ...,NaN,...,I cannot perform live web browsing or search t...,None,None,None,None,None,I cannot browse websites or access the interne...,I can't directly explore websites or access Wi...,None,None
75,Zolpidem,5732.0,"['zolpidem', '82626-48-0', 'Zolpidemum', 'Zolp...",Zolpidem,"['Ambien', 'Stilnox']",Zolpidem is a nonbenzodiazepine hypnotic that ...,"At therapeutic doses, zolpidem induces rapid s...",While effective for short-term treatment of in...,Zolpidem is a prescription drug and is classif...,NaN,...,I cannot directly explore or crawl websites in...,Zolpidem (commonly known as Ambien) is a non-b...,Zolpidem is a nonbenzodiazepine sedative-hypno...,Zolpidem (commonly known as Ambien) is a non-b...,"Zolpidem, commonly known as Ambien, appears in...",None,"After exploring Erowid.org, Zolpidem (commonly...",I cannot actively explore the Wikipedia webs